# Troubleshooting

Failures are common in scientific analysis, even for well tested workflows, due to
the novel nature and of scientific experiments and known artefacts that can occur.
Therefore, it is always to sanity-check results produced by workflows. When a problem
occurs in a multi-stage workflow it can be difficult to identify at which stage the
issue occurred. 

If running in debug mode (the default), runtime exceptions will be raised to the
call shell or debugger. However, when using asynchronous workers the errors will
be saved in `_error.pklz` pickle files inside the task's cache directory. For
example, given the following toy example

In [1]:
from pydra.design import python, workflow
from pydra.engine.submitter import Submitter

@python.define
def Add(x: float, y: float) -> float:
    return x + y

@python.define
def Subtract(x: float, y: float) -> float:
    return x - y

@python.define
def Divide(x: float, y: float) -> float:
    return x / y

@workflow.define
def UnsafeWorkflow(a: float, b: float, c: float) -> float:
    add = workflow.add(Add(x=a, y=b))
    divide = workflow.add(Divide(x=add.out, y=c))
    subtract = workflow.add(Subtract(x=divide.out, y=b))
    return subtract.out

# This workflow will fail because we are trying to divide by 0
failing_workflow = UnsafeWorkflow(a=10, b=5).split(c=[3, 2 ,0])

with Submitter() as sub:
    result = sub(failing_workflow)
    
if result.errored:
    print("Workflow failed with errors:\n" + str(result.errors))
else:
    print("Workflow completed successfully :)")

A newer version (0.25) of nipype/pydra is available. You are using 0.25.dev165+g61e6439b.d20250131


TypeError: Incorrect type for field in 'x' field of Divide interface : [15.0] is not of type <class 'float'> (and cannot be coerced to it)

## Stray lockfiles while debugging

During the execution of a task, a lockfile is generated to signify that a task is running. If a task/workflow is terminated by an interactive debugger these lockfiles can hang around.